In [ ]:
#https://www.datamanim.com/dataset/03_dataq/typetwo.html

#1. 분류

#1-1. 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)

In [ ]:
import pandas as pd
import numpy as np

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
#0. 데이터 로드
X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


#1. 이상치/결측치 처리 
X_train.isnull().sum(), X_test.isnull().sum(), y_train.isnull().sum()

#2. X,Y 나누기 / 필요없는 데이터 제거
X_test_id = X_test['CustomerId'].copy()
X_train = X_train.drop(['CustomerId'], axis = 1)
X_train = X_train.drop(['Surname'], axis = 1)
X_test = X_test.drop(['CustomerId'], axis = 1)
X_test = X_test.drop(['Surname'], axis = 1)
#y_train = y_train.drop(['CustomerId'], axis = 1)



#3. 라벨인코딩 + 더미변수화 = 원핫인코딩
from sklearn.preprocessing import LabelEncoder

X_train.select_dtypes('object')
label_encoding_features = ['Geography', 'Gender']
for feature in label_encoding_features:
    le = LabelEncoder()
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.fit_transform(X_test[feature])

categorical_features = ['NumOfProducts', 'HasCrCard', 'IsActiveMember']
for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_test[feature] = X_test[feature].astype('category')

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


#3. 파생변수 변경
# X_train['CreditScore'].value_counts()
X_test['CreditScore_qcut'] = pd.qcut(X_test['CreditScore'], 5, labels=False)
X_train['CreditScore_qcut'] = pd.qcut(X_train['CreditScore'], 5, labels=False)
# X_train['Age'].value_counts()
# X_train['Age_qcut'] = pd.qcut(X_train['Age'], 5, labels=False)
# X_test['Age_qcut'] = pd.qcut(X_test['Age'], 5, labels=False)



#4. 정규화, 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




#5. train/test split 
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train.iloc[:, 1], random_state=2022, stratify=y_train.iloc[:, 1])



#6. 모델 생성, 훈련
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model1 = LogisticRegression()
model1.fit(X_train, y_train)
predicted1 = model1.predict_proba(X_valid)

model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
predicted2 = model2.predict_proba(X_valid)

y_valid.shape, predicted1.shape, predicted2.shape


#7. 성능 검증
from sklearn.metrics import roc_auc_score

print("로지스틱 회귀분석 점수 : {}".format(roc_auc_score(y_valid, predicted1[:, 1])))
print("랜덤포레스트 점수 : {}".format(roc_auc_score(y_valid, predicted2[:, 1])))



#8. 최종 모델 적용 후 파일생성
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
predicted2 = model2.predict_proba(X_test)

result = pd.DataFrame({'CustomerId' : X_test_id, 'Exited' : predicted2[:, 1]})
result.to_csv('00000000.csv', index=False)


로지스틱 회귀분석 점수 : 0.8096536653016244
랜덤포레스트 점수 : 0.8438960669041871


#2) 이직여부 판단 데이터 (target: 1:이직, 0:이직x)

In [ ]:
import pandas as pd
import numpy as np 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


#0. 데이터 로드
import pandas as pd
#데이터 로드
X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
X_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")


#1. 결측치
X_test.isnull().sum()
X_train.isnull().sum()
y_train.isnull().sum()
#결측치를 확인해 본 결과 'gender', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job' 컬럼에 결측치가 있다. 

X_train['gender'] = X_train['gender'].fillna('Other')
X_test['gender'] = X_train['gender'].fillna('Other')

X_train['enrolled_university'] = X_train['enrolled_university'].fillna('no_enrollment')
X_test['enrolled_university'] = X_train['enrolled_university'].fillna('no_enrollment')

X_train['education_level'] = X_train['major_discipline'].fillna('Graduate')
X_test['education_level'] = X_test['major_discipline'].fillna('Graduate')

X_train['major_discipline'] = X_train['major_discipline'].fillna('STEM')
X_test['major_discipline'] = X_test['major_discipline'].fillna('STEM')

X_train['experience'] = X_train['experience'].fillna('0')
X_test['experience'] = X_test['experience'].fillna('0')

X_train['company_size'] = X_train['company_size'].fillna('10-')
X_test['company_size'] = X_test['company_size'].fillna('10-')

X_train['company_type'] = X_train['company_type'].fillna('other')
X_test['company_type'] = X_test['company_type'].fillna('other')

X_train['last_new_job'] = X_train['last_new_job'].fillna('never')
X_test['last_new_job'] = X_test['last_new_job'].fillna('never')


#2. X,Y 변수 정리
X_test_id = X_test['enrollee_id'] #고유ID
X_train.drop(columns=['enrollee_id', 'city'], inplace=True) #도시코드-범주 100개이상이라 분석 어려움
X_test.drop(columns=['enrollee_id', 'city'], inplace=True)


#for col in X_train.select_dtypes(object).columns.tolist():
#  print('-'*80)
#  print(col)
#  print(X_train[col].nunique())
#  print(X_train[col].value_counts())




#3. 원핫인코딩
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
X_train['training_hours'] = Scaler.fit_transform(X_train[['training_hours']])
X_test['training_hours'] = Scaler.fit_transform(X_test[['training_hours']])



#5. SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train.iloc[:,1], test_size=0.3)



#6. 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model1 = LogisticRegression()
model1.fit(X_train, y_train)
predicted1 = model1.predict_proba(X_test)

model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
predicted2 = model2.predict_proba(X_test)

y_valid.shape, predicted1.shape, predicted2.shape


#7. 성능 검증
from sklearn.metrics import roc_auc_score

print("로지스틱 회귀분석 점수 : {}".format(roc_auc_score(y_test, predicted1[:, 1])))
print("랜덤포레스트 점수 : {}".format(roc_auc_score(y_test, predicted2[:, 1])))



#8. 최종 모델 적용 후 파일생성
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
predicted2 = model2.predict_proba(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


로지스틱 회귀분석 점수 : 0.7769221502774135
랜덤포레스트 점수 : 0.7861270575086365


#3) e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

In [ ]:
import pandas as pd
#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")

#ID	Warehouse_block	Mode_of_Shipment	Customer_care_calls	Customer_rating	Cost_of_the_Product	Prior_purchases	Product_importance	Gender	Discount_offered	Weight_in_gms
#ID	Reached.on.Time_Y.N


#1. 결측치 확인 / 범주변수 확인
# print(x_train.isnull().sum(), '\n')
# print(y_train.isnull().sum())

#for i in data.select_dtypes(include=object):
    #print(i)
    #print(data[i].value_counts())
    #print(data.groupby(i)[y_train.columns[1]].mean(), '\n')

#for i in data.select_dtypes(exclude=object):
    #print(i)
    #print(data[i].value_counts())
    #print(data.groupby(i)[y_train.columns[1]].mean(), '\n')


#2. X,Y
x_train_id = x_train['ID']
x_test_id = x_test['ID']
x_train = x_train.drop(columns=['ID'], axis=1)
x_test =  x_test.drop(columns=['ID'], axis=1)
y_train = y_train['Reached.on.Time_Y.N']

x_train['Customer_care_calls'] = x_train['Customer_care_calls'].replace('$7', '7').astype(int)
x_test['Customer_care_calls'] = x_test['Customer_care_calls'].replace('$7', '7').astype(int)


#3. 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = ['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']
for i in cols:
    x_train[i] = le.fit_transform(x_train[i])
    x_test[i] = le.fit_transform(x_test[i])

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()

x_train['Cost_of_the_Product'] = mms.fit_transform(x_train[['Cost_of_the_Product']])
x_train['Discount_offered'] = mms.fit_transform(x_train[['Discount_offered']])
x_train['Weight_in_gms'] = mms.fit_transform(x_train[['Weight_in_gms']])

x_test['Cost_of_the_Product'] = mms.fit_transform(x_test[['Cost_of_the_Product']])
x_test['Discount_offered'] = mms.fit_transform(x_test[['Discount_offered']])
x_test['Weight_in_gms'] = mms.fit_transform(x_test[['Weight_in_gms']])

print(x_train.shape, y_train.shape, x_test.shape)


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

print(x_train.shape, y_train.shape, x_test.shape)

#6. 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model1 = LogisticRegression()
model1.fit(x_train, y_train)
predicted1 = model1.predict_proba(x_val)[:,1]

model2 = RandomForestClassifier()
model2.fit(x_train, y_train)
predicted2 = model2.predict_proba(x_val)[:,1]

predicted1.shape, predicted2.shape


#7. 성능 검증
from sklearn.metrics import roc_auc_score

print("로지스틱 회귀분석 점수 : {}".format(roc_auc_score(y_val, predicted1)))
print("랜덤포레스트 점수 : {}".format(roc_auc_score(y_val, predicted2)))

pred_test = model2.predict_proba(x_test)[:,1]
print(x_test_id.shape, pred_test.shape)

result = pd.DataFrame({'ID':x_test_id, 'Reached.on.Time_Y.N':pred_test})
result.to_csv('00000.csv', index=False)


(6598, 10) (6598,) (4401, 10)
(4618, 10) (4618,) (4401, 10)
로지스틱 회귀분석 점수 : 0.7209901453870683
랜덤포레스트 점수 : 0.754262048559853
(4401,) (4401,)


#4) 2018년도 성인의 건강검 진데이터 (종속변수 : 흡연상태 1- 흡연, 0-비흡연 )

In [13]:
import pandas as pd

#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv")

#display(x_train.head())
#display(y_train.head())

#1. 결측치
x_train.isnull().sum()
x_test.isnull().sum()


#2. x,y
x_test_id = x_test['ID']


#3. 원핫인코딩
x_train = pd.get_dummies(x_train, drop_first = True)
x_test = pd.get_dummies(x_test, drop_first = True)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

int_cols = x_train.select_dtypes(exclude=object).columns
#print(x_test.select_dtypes(exclude = object).columns)
mm.fit(x_train[int_cols])
x_train[int_cols] = mm.transform(x_train[int_cols])
x_test[int_cols] = mm.transform(x_test[int_cols])

#print(x_train.head())
#print(y_train.head())


x_train = pd.get_dummies(x_train.iloc[:,:-1])
x_test = pd.get_dummies(x_test)


#5. split
from sklearn.model_selection import train_test_split
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train['흡연상태'], random_state = 4442, test_size = 0.3)



#6. 모델
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state =42)
rf.fit(x_tr,y_tr)

pred1 = rf.predict(x_va)
pred_va = rf.predict_proba(x_va)[:,1]

from sklearn.metrics import roc_auc_score, classification_report, f1_score, confusion_matrix

print(f1_score(y_va, pred1))
print(roc_auc_score(y_va,pred_va))
print(classification_report(y_va, pred1))
print(confusion_matrix(y_va, pred1))


#7. 검증
#pred_test = rf.predict_proba(x_test)[:,1]


0.6693085000990687
0.8343314783086064
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      8383
           1       0.66      0.68      0.67      4983

    accuracy                           0.75     13366
   macro avg       0.73      0.74      0.73     13366
weighted avg       0.75      0.75      0.75     13366

[[6650 1733]
 [1605 3378]]


#5) 마케팅 응답 여부
- https://www.kaggle.com/datasets/kukuroo3/body-signal-of-smoking

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#0. 데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

#display(train.head(3))
#display(test.head(3))


#1. 결측치
train.isnull().sum()
test.isnull().sum()


#2. x,y
x_train = train.drop(columns = ['ID', 'y'])
x_test = test.drop(columns = ['ID'])
y_train = train['y']
y_train = y_train.map(lambda x : 0 if x=='no' else 1)

train_id = train['ID']
test_id = test['ID']

#print(x_train.select_dtypes(exclude = 'object').columns)


#3. 원핫인코딩
x_train = pd.get_dummies(x_train, drop_first = True)
x_test = pd.get_dummies(x_test, drop_first = True)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler

print(x_train.head())
mm = MinMaxScaler()
scaler = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

mm.fit(x_train[scaler])
x_train[scaler] = mm.transform(x_train[scaler])
x_test[scaler] = mm.transform(x_test[scaler])


#4. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,random_state = 42, stratify=y_train, test_size = 0.3)


#5. 모델
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train,y_train)

pred_tr = rf.predict_proba(x_train)[:,1]
pred_val = rf.predict_proba(x_val)[:,1]

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_train,pred_tr))
print(roc_auc_score(y_val,pred_val))


#6. 검증
pred_test = rf.predict_proba(x_test)[:,1]
pd.DataFrame({'ID':test_id, 'price': pred_test}).to_csv('0000.csv',index = False)
result = pd.read_csv('0000.csv')

KeyError: ignored

#6) 동작 분류
- https://www.kaggle.com/datasets/kukuroo3/motion-pose-classification-tabula-data

In [ ]:
import numpy as np 
import pandas as pd

x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv")

#1. 결측치 제거
x_train.isnull().sum()
y_train.isnull().sum()
x_test.isnull().sum()

for i in x_train.select_dtypes(exclude=object):
  x_train[i].value_counts()


#2. x,y
train_id = x_train['ID']
test_id = x_test['ID']
y_train = y_train['pose']
x_train = x_train.drop(['ID'], axis=1)
x_test = x_test.drop(['ID'], axis=1)


#3. 원핫인코딩
#모두 수치형이므로 생략


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict_proba(x_val)
print(roc_auc_score(y_val, lr_pred[:, 1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf_pred = rf.predict_proba(x_val)
print(roc_auc_score(y_val, rf_pred[:, 1]))


#7. 결론
rf.fit(x_train, y_train)
result_pred = rf.predict_proba(x_test)

result = pd.DataFrame({'ID':test_id, 'pose':result_pred[:,1]})
result.to_csv('00000.csv', index=False)

check=pd.read_csv('00000.csv')
print(check.head())

0.5653825441001756
0.9999897226344587
   ID  pose
0   1  0.81
1   3  0.84
2   8  0.67
3  10  0.76
4  17  0.09


#7) 뇌졸증 여부
- https://www.kaggle.com/datasets/kukuroo3/stoke-prediction-dataset?select=train.csv

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


#1. 결측치
train.isnull().sum() #bmi 52
test.isnull().sum() #bmi 57

train.head()
#id	/gender	age	hypertension	heart_disease	/ever_married	/work_type	/Residence_type	avg_glucose_level	bmi	/smoking_status	stroke
test.head()

for i in train.select_dtypes(exclude=object):
  train[i].value_counts()

for i in train.select_dtypes(include=object):
  train[i].value_counts()


#2. X,Y 나누기
train['age'] = train['age'].replace('*82', '82').astype(int)
train['gender'] = train['gender'].replace('Other', 'Male')
train.bmi = train.bmi.fillna(train.bmi.median())
test.bmi = test.bmi.fillna(test.bmi.median())
x_train_id = train['id']
x_test_id = test['id']

x_train = train.drop(columns = ['id', 'stroke'], axis=1)
x_test = test.drop(columns = ['id'], axis=1)
y_train = train['stroke']


#3. 더미,원핫
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)
print(roc_auc_score(y_val, pred1[:, 1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:, 1]))


#7. 결과
rf.fit(x_train, y_train)
pred = rf.predict_proba(x_test)


result = pd.DataFrame({'ID': x_test_id, 'result': pred[:,1]})
result.to_csv('00000.csv', index=False)

a = pd.read_csv('00000.csv')
a.head()


0.8119277206061097
0.81310266591038


,ID,result
0,47472,0.16
1,36841,0.33
2,3135,0.22
3,65218,0.00
4,1847,0.01


#8) 자동차 보험 가입 예측 (종속변수 Response: 1 : 가입 , 0 :미가입)

In [ ]:
import pandas as pd

#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv")

#display(x_train.head())
#	ID	/Gender	Age	Driving_License	Region_Code	Previously_Insured	/Vehicle_Age	/Vehicle_Damage	Annual_Premium	Policy_Sales_Channel	Vintage	/id
#display(y_train.head())
#ID	Response


#1. 결측치제거
x_train.isnull().sum()
y_train.isnull().sum()
x_test.isnull().sum()

for i in x_train.select_dtypes(exclude=object):
  (x_train[i].value_counts())

for i in x_train.select_dtypes(include=object):
  (x_train[i].value_counts())

#x_train.info()
#Gender Vehicle_Age Vehicle_Damage        


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop(['ID', 'id'], axis=1)
x_test = x_test.drop(['ID', 'id'], axis=1)
y_train = y_train['Response']


#3. 라벨 / 원핫
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
for i in cols:
    x_train[i] = le.fit_transform(x_train[i])
    x_test[i] = le.fit_transform(x_test[i])

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)



#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#5. SPLIT
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)
print(roc_auc_score(y_val, pred1[:,1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:,1]))


#7. 예측
pred3 = rf.predict_proba(x_test)
result = pd.DataFrame({'ID': x_test_id, 'pred': pred3[:,1]})

result.to_csv('0000.csv', index=False)


0.8239510100770036
0.837710053455559


#9) 비행탑승 경험 만족도 데이터

In [ ]:
import pandas as pd

#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")


#display(x_train.head())
#display(y_train.head())


#1. 결측치제거
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())
#print(x_test.isnull().sum())

#for i in x_train.select_dtypes(exclude=object):
#  print(x_train[i].value_counts())

#for i in x_train.select_dtypes(include=object):
#  print(x_train[i].value_counts())


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop(['ID', 'id'], axis=1)
x_test = x_test.drop(['ID', 'id'], axis=1)
y_train = y_train['satisfaction']

x_train['Arrival Delay in Minutes'] = x_train['Arrival Delay in Minutes'].fillna(0)
x_test['Arrival Delay in Minutes'] = x_test['Arrival Delay in Minutes'].fillna(0)


#3. 라벨 / 원핫
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'Arrival Delay in Minutes']
for i in cols:
  x_train[i] = le.fit_transform(x_train[i])
  x_test[i] = le.fit_transform(x_test[i])

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#5. SPLIT
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)
print(roc_auc_score(y_val, pred1[:,1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:,1]))

#7. 예측
pred3 = rf.predict_proba(x_test)
result = pd.DataFrame({'ID': x_test_id, 'pred': pred3[:,1]})

result.to_csv('0000.csv', index=False)



0.9262898983891156
0.9932132658172367


#10) 수질 음용성 여부 (Potablillity 컬럼 : 0 ,1 )

In [ ]:
import pandas as pd

#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_test.csv")


#display(x_train.head())
#ID	ph	Hardness	Solids	Chloramines	Sulfate	Conductivity	Organic_carbon	Trihalomethanes	Turbidity
#display(y_train.head())
#ID	Potability


#1. 결측치제거
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())
#print(x_test.isnull().sum())

#for i in x_train.select_dtypes(exclude=object):
#  print(x_train[i].value_counts())

#for i in x_train.select_dtypes(include=object):
#  print(x_train[i].value_counts())


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop(['ID'], axis=1)
x_test = x_test.drop(['ID'], axis=1)
y_train = y_train['Potability']

x_train['ph'] = x_train['ph'].fillna(x_train.ph.median())
x_train['Sulfate'] = x_train['Sulfate'].fillna(x_train.Sulfate.median())
x_train['Trihalomethanes'] = x_train['Trihalomethanes'].fillna(x_train.Trihalomethanes.median())

x_test['ph'] = x_test['ph'].fillna(x_train.ph.median())
x_test['Sulfate'] = x_test['Sulfate'].fillna(x_train.Sulfate.median())
x_test['Trihalomethanes'] = x_test['Trihalomethanes'].fillna(x_train.Trihalomethanes.median())


#3. 라벨 / 원핫
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = ['ph', 'Sulfate', 'Trihalomethanes']
for i in cols:
  x_train[i] = le.fit_transform(x_train[i])
  x_test[i] = le.fit_transform(x_test[i])

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#5. SPLIT
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)
print(roc_auc_score(y_val, pred1[:,1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:,1]))


#7. 예측
pred3 = rf.predict_proba(x_test)
result = pd.DataFrame({'ID': x_test_id, 'pred': pred3[:,1]})

result.to_csv('0000.csv', index=False)

0.47466559675763154
0.6582116651819413


#11) 투약하는 약을 분류 (종속변수 :Drug)

In [ ]:
import pandas as pd

#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/x_test.csv")


#display(x_train.head())
#	ID	Age	Sex	BP	Cholesterol	Na_to_K
#display(y_train.head())
#	ID	Drug


#1. 결측치제거
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())
#print(x_test.isnull().sum())

#for i in x_train.select_dtypes(exclude=object):
# print(x_train[i].value_counts())

#for i in x_train.select_dtypes(include=object):
#  print(x_train[i].value_counts())


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop(['ID'], axis=1)
x_test = x_test.drop(['ID'], axis=1)
y_train = y_train['Drug']


#3. 라벨 / 원핫
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = ['Sex', 'BP', 'Cholesterol']
for i in cols:
  x_train[i] = le.fit_transform(x_train[i])
  x_test[i] = le.fit_transform(x_test[i])

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)



#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)



#5. SPLIT
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)

print(roc_auc_score(y_val, pred1, multi_class='ovr'))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2, multi_class='ovr'))


#7. 예측
pred3 = rf.predict_proba(x_test)
result = pd.DataFrame({'ID': x_test_id, 'pred': pred3[:,1]})

result.to_csv('0000.csv', index=False)

0.9690298074019005
1.0


#12) 사기회사 분류 (종속변수 : Risk 1: 사기 , 0 : 정상)

In [ ]:
import pandas as pd

#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_test.csv")


#display(x_train.head())
#display(y_train.head())

#1. 결측치제거
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())
#print(x_test.isnull().sum())

#for i in x_train.select_dtypes(exclude=object):
# print(x_train[i].value_counts())

#for i in x_train.select_dtypes(include=object):
#  print(x_train[i].value_counts())


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop(['ID'], axis=1)
x_test = x_test.drop(['ID'], axis=1)
y_train = y_train['Risk']

x_train['Money_Value'] = x_train['Money_Value'].fillna(0)
x_test['Money_Value'] = x_test['Money_Value'].fillna(0)


x_train[x_train['LOCATION_ID']=='SAFIDON'] = '8'
x_train[x_train['LOCATION_ID']=='LOHARU'] = '8'
x_train[x_train['LOCATION_ID']=='NUH'] = '8'

x_test[x_test['LOCATION_ID']=='SAFIDON'] = '8'
x_test[x_test['LOCATION_ID']=='LOHARU'] = '8'
x_test[x_test['LOCATION_ID']=='NUH'] = '8'


#3. 라벨 / 원핫
#패스

#x_train = pd.get_dummies(x_train)
#y_train = pd.get_dummies(y_train)



#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)



#5. SPLIT
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=44, stratify=y_train, test_size=0.3)


#6. 모델 생성
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)

print(roc_auc_score(y_val, pred1[:,1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:,1]))


#7. 예측
pred3 = rf.predict_proba(x_test)
result = pd.DataFrame({'ID': x_test_id, 'pred': pred3[:,1]})

result.to_csv('0000.csv', index=False)

0.9741786883258576
1.0


#13)  당뇨여부 판단하기 (종속변수 Outcome : 1 당뇨 , 0 :정상)

In [ ]:
import pandas as pd

#0. 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_test.csv")


#display(x_train.head())
#ID	Pregnancies	Glucose	BloodPressure	SkinThickness	Insulin	BMI	DiabetesPedigreeFunction	Age
#display(y_train.head())
#ID	Outcome


#1. 결측치
x_train.isnull().sum()
x_test.isnull().sum()
y_train.isnull().sum()

for i in x_train.select_dtypes(exclude=object):
  x_train[i].value_counts()

x_train.head()


#2. x,y
x_train_id = x_train['ID']
x_test_id = x_test['ID']

x_train = x_train.drop('ID', axis=1)
x_test = x_test.drop('ID', axis=1)
y_train = y_train['Outcome']


#3. 원핫인코딩, 라벨
#패스

#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


#5. split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=42, test_size=0.3, stratify=y_train)


#6. 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)
pred1 = lr.predict_proba(x_val)
print(roc_auc_score(y_val, pred1[:, 1]))

rf = RandomForestClassifier()
rf.fit(x_train, y_train)
pred2 = rf.predict_proba(x_val)
print(roc_auc_score(y_val, pred2[:, 1]))


#7. 결과 출력
pred3 = rf.predict_proba(x_test)

result = pd.DataFrame({'ID':x_test_id, 'result':pred3[:,1]})
result.to_csv('0000.csv', index=False)

read = pd.read_csv('0000.csv')
read.head()

0.8383264462809917
0.8558884297520661


,ID,result
0,13,0.32
1,18,0.17
2,29,0.27
3,33,0.52
4,34,0.12
